In [1]:
import sys
sys.path.append("/home/oldrain123/IMBALANCED_CLASSIFICATION/MOMs")
sys.path.append("/home/oldrain123/IMBALANCED_CLASSIFICATION/boost")
sys.path.append('/home/oldrain123/IMBALANCED_CLASSIFICATION/')
sys.path.append('/home/oldrain123/IMBALANCED_CLASSIFICATION/SMOTE_variants/')

In [2]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, matthews_corrcoef, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, RandomOverSampler
from imblearn.metrics import geometric_mean_score
from collections import Counter
from moms_generate import apply_transformation
from moms_losses import MMD_est_torch
from boost import AdaBoostClassifier, SMOTEBoost, RUSBoost, OUBoost
from moms_utils import set_seed
from sklearn.svm import SVC
from SMOTE_variants.sm_variants.oversampling.mwmote import MWMOTE

In [3]:
device = "cuda:1"
print(f"Using device: {device}")

Using device: cuda:1


In [4]:
def run_experiment(data, device, h_dim, num_layers, beta, lr, n_runs=10, n_splits=10, save_path = '/results', data_name = 'wine'):
    """
    Run experiments on the provided dataset.

    Parameters:
    - data: pandas DataFrame with the last column as labels.
    - n_runs: Number of repeated experiments.
    - n_splits: Number of splits for Stratified K-Fold.

    Returns:
    - final_results: Dictionary containing averaged metrics for all methods.
    """
    X = data.iloc[:, :-1].values
    Y = np.where(data.iloc[:, -1].values == 'negative', 0, 1)  # Convert labels
    print(f"X shape: {X.shape}")
    print(f"Class distribution: {Counter(Y)}")

    # Parameters
    n_epochs, h = 2000, 1.0

    # Initialize final results storage
    final_results = {
        method: {"AUC": [], "G-Mean": [], "MCC": [], "F1-score": []}
        for method in ["AdaBoost", "SMOTEBoost", "RUSBoost", "OUBoost", "SVM", "SMOTE", "ADASYN", "bSMOTE", "ROS", "MWMOTE", "Trans(Direct)"]
    }

    for run in range(n_runs):
        kf = StratifiedKFold(n_splits=n_splits, random_state=1203 + run, shuffle=True)
        print(f"\nStarting experiment {run + 1}/{n_runs}")
        results = {
            method: {"AUC": [], "G-Mean": [], "MCC": [], "F1-score": []}
            for method in ["AdaBoost", "SMOTEBoost", "RUSBoost", "OUBoost", "SVM", "SMOTE", "ADASYN", "bSMOTE", "ROS", "MWMOTE", "Trans(Direct)"]
        }

        for fold, (train_index, test_index) in enumerate(kf.split(X, Y)):
            # if fold < 3:
            #     continue
            print(f"  Fold {fold + 1}/{n_splits} - Experiment {run + 1}/{n_runs}")
            seed = 1203 + fold + 10 * run
            set_seed(seed)
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = np.ascontiguousarray(X_train, dtype=np.float64)
            X_test = np.ascontiguousarray(X_test, dtype=np.float64)

            X_maj = X_train[Y_train == 0]
            X_min = X_train[Y_train == 1]

            input_dim = X_train.shape[1]

            # Apply transformations
            X_maj_direct, X_min_direct, X_trans_direct = apply_transformation(
                X_maj,
                X_min,
                in_dim=input_dim,
                h_dim=h_dim,
                num_layers=num_layers,
                loss_fn=MMD_est_torch,
                device=device,
                method='direct',
                # selection="overlap",
                n_epochs=n_epochs,
                h=h,
                beta=beta,
                lr=lr,
                seed=seed,
                batch_size=128,
                k=3,
                undersample=False
            )

            datasets = {
                "SVM": (X_train, Y_train),
                "Boost": (X_train, Y_train),
                "SMOTE": SMOTE(random_state=seed).fit_resample(X_train, Y_train),
                "ADASYN": ADASYN(random_state=seed).fit_resample(X_train, Y_train),
                "bSMOTE": BorderlineSMOTE(random_state=seed).fit_resample(X_train, Y_train),
                "ROS": RandomOverSampler(random_state=seed).fit_resample(X_train, Y_train),
                "MWMOTE": MWMOTE(random_state=seed).sample(X_train, Y_train),
                "Trans(Direct)": (
                    np.vstack((X_maj_direct, X_min_direct, X_trans_direct)),
                    np.hstack((np.zeros(len(X_maj_direct)), np.ones(len(X_min_direct) + len(X_trans_direct)))),
                ),
            }

            for method, (X_resampled, Y_resampled) in datasets.items():
                if method == "Boost":
                    # AdaBoost
                    model_ada = AdaBoostClassifier(
                        DecisionTreeClassifier(max_depth=5),
                        n_estimators=100,
                        algorithm="SAMME",
                        learning_rate=0.1,
                        random_state=seed,
                    )
                    model_ada.fit(X_resampled, Y_resampled)
                    predictions_ada = model_ada.predict(X_test)
                    proba_ada = model_ada.predict_proba(X_test)[:, 1]
                    results["AdaBoost"]["AUC"].append(roc_auc_score(Y_test, proba_ada))
                    results["AdaBoost"]["G-Mean"].append(
                        np.mean(geometric_mean_score(Y_test, predictions_ada, average=None))
                    )
                    results["AdaBoost"]["MCC"].append(matthews_corrcoef(Y_test, predictions_ada))
                    results['AdaBoost']['F1-score'].append(f1_score(Y_test, predictions_ada))

                    # SMOTEBoost
                    classification_smote = SMOTEBoost(
                        learning_rate=0.1, n_samples=5, n_estimators=100, random_state=seed
                    )
                    classification_smote.fit(X_resampled, Y_resampled)
                    y_pred_smote = classification_smote.predict(X_test)
                    proba_smote = classification_smote.predict_proba(X_test)[:, 1]
                    results["SMOTEBoost"]["AUC"].append(roc_auc_score(Y_test, proba_smote))
                    results["SMOTEBoost"]["G-Mean"].append(
                        np.mean(geometric_mean_score(Y_test, y_pred_smote, average=None))
                    )
                    results["SMOTEBoost"]["MCC"].append(matthews_corrcoef(Y_test, y_pred_smote))
                    results['SMOTEBoost']['F1-score'].append(f1_score(Y_test, y_pred_smote))

                    # RUSBoost
                    classification_rusboost = RUSBoost(
                        learning_rate=0.1, n_samples=5, n_estimators=100, random_state=seed
                    )
                    classification_rusboost.fit(X_resampled, Y_resampled)
                    y_pred_rus = classification_rusboost.predict(X_test)
                    proba_rus = classification_rusboost.predict_proba(X_test)[:, 1]
                    results["RUSBoost"]["AUC"].append(roc_auc_score(Y_test, proba_rus))
                    results["RUSBoost"]["G-Mean"].append(
                        np.mean(geometric_mean_score(Y_test, y_pred_rus, average=None))
                    )
                    results["RUSBoost"]["MCC"].append(matthews_corrcoef(Y_test, y_pred_rus))
                    results['RUSBoost']['F1-score'].append(f1_score(Y_test, y_pred_rus))

                    # OUBoost
                    classification_ouboost = OUBoost(
                        learning_rate=0.1, n_samples=5, n_estimators=100, random_state=seed
                    )
                    classification_ouboost.fit(X_resampled, Y_resampled)
                    y_pred_ouboost = classification_ouboost.predict(X_test)
                    proba_ouboost = classification_ouboost.predict_proba(X_test)[:, 1]
                    results["OUBoost"]["AUC"].append(roc_auc_score(Y_test, proba_ouboost))
                    results["OUBoost"]["G-Mean"].append(
                        np.mean(geometric_mean_score(Y_test, y_pred_ouboost, average=None))
                    )
                    results["OUBoost"]["MCC"].append(matthews_corrcoef(Y_test, y_pred_ouboost))
                    results['OUBoost']['F1-score'].append(f1_score(Y_test, y_pred_ouboost))
                else:
                    svm = SVC(kernel='rbf', probability=True, random_state=seed)
                    svm.fit(X_resampled, Y_resampled)

                    # Predict on the test data
                    y_pred = svm.predict(X_test)
                    y_pred_prob = svm.predict_proba(X_test)[:, 1]

                    # xgb = XGBClassifier(n_estimators=100, max_depth=5, random_state=1203)
                    # xgb.fit(X_resampled, Y_resampled)
                    # y_pred = xgb.predict(X_test)
                    # y_pred_prob = xgb.predict_proba(X_test)[:, 1]
                    # Calculate performance metrics
                    results[method]["AUC"].append(roc_auc_score(Y_test, y_pred_prob))
                    results[method]["G-Mean"].append(
                        np.mean(geometric_mean_score(Y_test, y_pred, average=None))
                    )
                    results[method]["MCC"].append(matthews_corrcoef(Y_test, y_pred))
                    results[method]["F1-score"].append(f1_score(Y_test, y_pred))

            # Print fold-wise results for monitoring
            print(f"    Intermediate Fold Results for Fold {fold + 1}:")
            for method, metrics in results.items():
                print(f"      {method}: AUC = {np.mean(metrics['AUC']):.4f}, "
                      f"G-Mean = {np.mean(metrics['G-Mean']):.4f}, "
                      f"MCC = {np.mean(metrics['MCC']):.4f}, "
                      f"F1-score = {np.mean(metrics['F1-score']):.4f}")

        # Aggregate results across folds for this experiment
        for method, metrics in results.items():
            for metric, values in metrics.items():
                final_results[method][metric].append(np.round(np.mean(values), 4))

    # Print final averaged results
    print("\nFinal Averaged Results Across Experiments:")
    for method, metrics in final_results.items():
        print(f"  {method}:")
        for metric, values in metrics.items():
            print(f"    {metric}: {np.mean(values):.4f}")

    # Convert final_results to a pandas DataFrame
    result_data = {
        "Method": [],
        "Metric": [],
        "Value": [],
    }

    for method, metrics in final_results.items():
        for metric, values in metrics.items():
            avg_value = np.mean(values) if values else "N/A"
            result_data["Method"].append(method)
            result_data["Metric"].append(metric)
            result_data["Value"].append(avg_value)

    results_df = pd.DataFrame(result_data)

    # Save results as CSV
    os.makedirs(save_path, exist_ok=True)
    save_file = os.path.join(save_path, f"{data_name}_final_results.csv")
    results_df.to_csv(save_file, index=False)
    print(f"\nFinal results saved to {save_file}")
    
    return final_results


In [5]:
# Save path 
data_path = "/data4/oldrain123/oldrain123/dataset/"
save_path = "/data4/oldrain123/oldrain123/results/real_results"

In [ ]:
# pima
data_name = 'pima'
dataframe = pd.read_csv(f"{data_path}{data_name}.dat")
results = run_experiment(dataframe, device, h_dim=256, num_layers=10, beta=0.3, lr = 0.001, save_path=save_path, data_name = data_name)

for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for metric, values in metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"  {metric}: {mean_value:.4f} ± {std_value:.4f}")

X shape: (767, 8)
Class distribution: Counter({np.int64(0): 500, np.int64(1): 267})

Starting experiment 1/10
  Fold 1/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.46145, Reg Loss: 0.17139
Epoch 200/2000, Avg Loss: 0.41218, Reg Loss: 0.16937
Epoch 300/2000, Avg Loss: 0.39246, Reg Loss: 0.17105
Epoch 400/2000, Avg Loss: 0.37651, Reg Loss: 0.17399
Epoch 500/2000, Avg Loss: 0.36113, Reg Loss: 0.17034
Epoch 600/2000, Avg Loss: 0.43490, Reg Loss: 0.18842
Epoch 700/2000, Avg Loss: 0.33712, Reg Loss: 0.17302
Epoch 800/2000, Avg Loss: 0.32110, Reg Loss: 0.16783
Epoch 900/2000, Avg Loss: 0.31151, Reg Loss: 0.16752
Epoch 1000/2000, Avg Loss: 0.30176, Reg Loss: 0.16772
Epoch 1100/2000, Avg Loss: 0.28856, Reg Loss: 0.16521
Epoch 1200/2000, Avg Loss: 0.28817, Reg Loss: 0.16394
Epoch 1300/2000, Avg Loss: 0.28428, Reg Loss: 0.16517
Epoch 1400/2000, Avg Loss: 0.26735, Reg Loss: 0.16300
Epoch 1500/2000, Avg Loss: 0.26442, Reg Loss: 0.16165
Epoch 1600/2000, Avg Loss: 0.26801, Reg Loss: 0.16093
Epoch

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 1:
      AdaBoost: AUC = 0.7533, G-Mean = 0.6360, MCC = 0.3042, F1-score = 0.5385
      SMOTEBoost: AUC = 0.8630, G-Mean = 0.7688, MCC = 0.5437, F1-score = 0.7037
      RUSBoost: AUC = 0.6800, G-Mean = 0.4422, MCC = 0.1347, F1-score = 0.5333
      OUBoost: AUC = 0.8552, G-Mean = 0.7483, MCC = 0.5112, F1-score = 0.6792
      SVM: AUC = 0.8622, G-Mean = 0.7221, MCC = 0.4992, F1-score = 0.6531
      SMOTE: AUC = 0.8185, G-Mean = 0.7587, MCC = 0.4975, F1-score = 0.6885
      ADASYN: AUC = 0.8393, G-Mean = 0.7765, MCC = 0.5317, F1-score = 0.7097
      bSMOTE: AUC = 0.8259, G-Mean = 0.7552, MCC = 0.4917, F1-score = 0.6875
      ROS: AUC = 0.8548, G-Mean = 0.7503, MCC = 0.4848, F1-score = 0.6780
      MWMOTE: AUC = 0.8422, G-Mean = 0.7611, MCC = 0.5076, F1-score = 0.6970
      Trans(Direct): AUC = 0.8630, G-Mean = 0.7659, MCC = 0.5115, F1-score = 0.6984
  Fold 2/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.5

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 2:
      AdaBoost: AUC = 0.7926, G-Mean = 0.7024, MCC = 0.4240, F1-score = 0.6211
      SMOTEBoost: AUC = 0.8417, G-Mean = 0.7541, MCC = 0.5152, F1-score = 0.6852
      RUSBoost: AUC = 0.6998, G-Mean = 0.4950, MCC = 0.2481, F1-score = 0.5700
      OUBoost: AUC = 0.8422, G-Mean = 0.7378, MCC = 0.4949, F1-score = 0.6665
      SVM: AUC = 0.8600, G-Mean = 0.7440, MCC = 0.5312, F1-score = 0.6795
      SMOTE: AUC = 0.8311, G-Mean = 0.7688, MCC = 0.5188, F1-score = 0.7002
      ADASYN: AUC = 0.8367, G-Mean = 0.7906, MCC = 0.5590, F1-score = 0.7258
      bSMOTE: AUC = 0.8263, G-Mean = 0.7939, MCC = 0.5663, F1-score = 0.7308
      ROS: AUC = 0.8533, G-Mean = 0.7686, MCC = 0.5186, F1-score = 0.6996
      MWMOTE: AUC = 0.8363, G-Mean = 0.8022, MCC = 0.5848, F1-score = 0.7419
      Trans(Direct): AUC = 0.8581, G-Mean = 0.7905, MCC = 0.5593, F1-score = 0.7263
  Fold 3/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.5

...

predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 8:
      AdaBoost: AUC = 0.8381, G-Mean = 0.7231, MCC = 0.4724, F1-score = 0.6491
      SMOTEBoost: AUC = 0.8219, G-Mean = 0.7348, MCC = 0.4646, F1-score = 0.6613
      RUSBoost: AUC = 0.7143, G-Mean = 0.4151, MCC = 0.2471, F1-score = 0.5838
      OUBoost: AUC = 0.8247, G-Mean = 0.7189, MCC = 0.4486, F1-score = 0.6419
      SVM: AUC = 0.8373, G-Mean = 0.6902, MCC = 0.4580, F1-score = 0.6131
      SMOTE: AUC = 0.8277, G-Mean = 0.7369, MCC = 0.4683, F1-score = 0.6631
      ADASYN: AUC = 0.8280, G-Mean = 0.7433, MCC = 0.4780, F1-score = 0.6726
      bSMOTE: AUC = 0.8190, G-Mean = 0.7325, MCC = 0.4535, F1-score = 0.6601
      ROS: AUC = 0.8321, G-Mean = 0.7326, MCC = 0.4606, F1-score = 0.6590
      MWMOTE: AUC = 0.8221, G-Mean = 0.7300, MCC = 0.4526, F1-score = 0.6562
      Trans(Direct): AUC = 0.8419, G-Mean = 0.7412, MCC = 0.4729, F1-score = 0.6689
  Fold 9/10 - Experiment 10/10
Epoch 100/2000, Avg Loss: 0.

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 9:
      AdaBoost: AUC = 0.8245, G-Mean = 0.7122, MCC = 0.4517, F1-score = 0.6347
      SMOTEBoost: AUC = 0.8163, G-Mean = 0.7349, MCC = 0.4632, F1-score = 0.6606
      RUSBoost: AUC = 0.7214, G-Mean = 0.4224, MCC = 0.2479, F1-score = 0.5813
      OUBoost: AUC = 0.8181, G-Mean = 0.7150, MCC = 0.4397, F1-score = 0.6364
      SVM: AUC = 0.8307, G-Mean = 0.6802, MCC = 0.4349, F1-score = 0.5994
      SMOTE: AUC = 0.8211, G-Mean = 0.7258, MCC = 0.4455, F1-score = 0.6497
      ADASYN: AUC = 0.8212, G-Mean = 0.7406, MCC = 0.4721, F1-score = 0.6696
      bSMOTE: AUC = 0.8133, G-Mean = 0.7302, MCC = 0.4484, F1-score = 0.6573
      ROS: AUC = 0.8252, G-Mean = 0.7185, MCC = 0.4330, F1-score = 0.6413
      MWMOTE: AUC = 0.8193, G-Mean = 0.7260, MCC = 0.4436, F1-score = 0.6514
      Trans(Direct): AUC = 0.8347, G-Mean = 0.7297, MCC = 0.4496, F1-score = 0.6549
  Fold 10/10 - Experiment 10/10
Epoch 100/2000, Avg Loss: 0

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 10:
      AdaBoost: AUC = 0.8250, G-Mean = 0.7113, MCC = 0.4577, F1-score = 0.6349
      SMOTEBoost: AUC = 0.8179, G-Mean = 0.7298, MCC = 0.4538, F1-score = 0.6538
      RUSBoost: AUC = 0.7286, G-Mean = 0.4193, MCC = 0.2409, F1-score = 0.5770
      OUBoost: AUC = 0.8193, G-Mean = 0.7176, MCC = 0.4456, F1-score = 0.6394
      SVM: AUC = 0.8310, G-Mean = 0.6833, MCC = 0.4459, F1-score = 0.6046
      SMOTE: AUC = 0.8236, G-Mean = 0.7297, MCC = 0.4518, F1-score = 0.6537
      ADASYN: AUC = 0.8241, G-Mean = 0.7449, MCC = 0.4792, F1-score = 0.6739
      bSMOTE: AUC = 0.8175, G-Mean = 0.7348, MCC = 0.4572, F1-score = 0.6623
      ROS: AUC = 0.8275, G-Mean = 0.7229, MCC = 0.4399, F1-score = 0.6460
      MWMOTE: AUC = 0.8227, G-Mean = 0.7318, MCC = 0.4535, F1-score = 0.6575
      Trans(Direct): AUC = 0.8343, G-Mean = 0.7322, MCC = 0.4542, F1-score = 0.6572

Final Averaged Results Across Experiments:
  AdaBoost:
  

In [10]:
# glass-0-1-2-3_vs_4-5-6
data_name = 'glass-0-1-2-3_vs_4-5-6'
dataframe = pd.read_csv(f"{data_path}{data_name}.dat")
results = run_experiment(dataframe, device, h_dim=512, num_layers=10, beta=0.1, lr = 0.001, save_path=save_path, data_name = data_name)

for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for metric, values in metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"  {metric}: {mean_value:.4f} ± {std_value:.4f}")

X shape: (213, 9)
Class distribution: Counter({np.int64(0): 162, np.int64(1): 51})

Starting experiment 1/10
  Fold 1/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.27251, Reg Loss: 0.31265
Epoch 200/2000, Avg Loss: 0.19615, Reg Loss: 0.31696
Epoch 300/2000, Avg Loss: 0.16760, Reg Loss: 0.30553
Epoch 400/2000, Avg Loss: 0.14533, Reg Loss: 0.30166
Epoch 500/2000, Avg Loss: 0.15223, Reg Loss: 0.30075
Epoch 600/2000, Avg Loss: 0.12104, Reg Loss: 0.31112
Epoch 700/2000, Avg Loss: 0.12560, Reg Loss: 0.31115
Epoch 800/2000, Avg Loss: 0.11309, Reg Loss: 0.29660
Epoch 900/2000, Avg Loss: 0.11050, Reg Loss: 0.31387
Epoch 1000/2000, Avg Loss: 0.11335, Reg Loss: 0.29589
Epoch 1100/2000, Avg Loss: 0.10395, Reg Loss: 0.29034
Epoch 1200/2000, Avg Loss: 0.10648, Reg Loss: 0.28794
Epoch 1300/2000, Avg Loss: 0.09280, Reg Loss: 0.28364
Epoch 1400/2000, Avg Loss: 0.10498, Reg Loss: 0.28982
Epoch 1500/2000, Avg Loss: 0.08492, Reg Loss: 0.28056
Epoch 1600/2000, Avg Loss: 0.08725, Reg Loss: 0.27980
Epoch 

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 1:
      AdaBoost: AUC = 0.9882, G-Mean = 0.9701, MCC = 0.8856, F1-score = 0.9091
      SMOTEBoost: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      RUSBoost: AUC = 1.0000, G-Mean = 0.9393, MCC = 0.7939, F1-score = 0.8333
      OUBoost: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      SVM: AUC = 1.0000, G-Mean = 0.8944, MCC = 0.8692, F1-score = 0.8889
      SMOTE: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      ADASYN: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      bSMOTE: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      ROS: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      MWMOTE: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      Trans(Direct): AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
  Fold 2/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.2

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 2:
      AdaBoost: AUC = 0.9941, G-Mean = 0.9323, MCC = 0.8774, F1-score = 0.8990
      SMOTEBoost: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      RUSBoost: AUC = 0.9706, G-Mean = 0.7122, MCC = 0.5248, F1-score = 0.6341
      OUBoost: AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
      SVM: AUC = 0.9941, G-Mean = 0.8944, MCC = 0.8692, F1-score = 0.8889
      SMOTE: AUC = 1.0000, G-Mean = 0.9472, MCC = 0.9346, F1-score = 0.9444
      ADASYN: AUC = 1.0000, G-Mean = 0.9472, MCC = 0.9346, F1-score = 0.9444
      bSMOTE: AUC = 0.9941, G-Mean = 0.9472, MCC = 0.9346, F1-score = 0.9444
      ROS: AUC = 1.0000, G-Mean = 0.9472, MCC = 0.9346, F1-score = 0.9444
      MWMOTE: AUC = 1.0000, G-Mean = 0.9472, MCC = 0.9346, F1-score = 0.9444
      Trans(Direct): AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
  Fold 3/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.2

...

predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 8:
      AdaBoost: AUC = 0.9143, G-Mean = 0.8929, MCC = 0.7987, F1-score = 0.8386
      SMOTEBoost: AUC = 0.9645, G-Mean = 0.9371, MCC = 0.8664, F1-score = 0.8940
      RUSBoost: AUC = 0.9636, G-Mean = 0.7619, MCC = 0.5640, F1-score = 0.6637
      OUBoost: AUC = 0.9788, G-Mean = 0.9107, MCC = 0.8128, F1-score = 0.8505
      SVM: AUC = 0.9799, G-Mean = 0.9086, MCC = 0.8354, F1-score = 0.8647
      SMOTE: AUC = 0.9799, G-Mean = 0.9135, MCC = 0.8286, F1-score = 0.8587
      ADASYN: AUC = 0.9829, G-Mean = 0.9322, MCC = 0.8600, F1-score = 0.8874
      bSMOTE: AUC = 0.9860, G-Mean = 0.9322, MCC = 0.8600, F1-score = 0.8874
      ROS: AUC = 0.9859, G-Mean = 0.9322, MCC = 0.8600, F1-score = 0.8874
      MWMOTE: AUC = 0.9844, G-Mean = 0.9454, MCC = 0.8765, F1-score = 0.9013
      Trans(Direct): AUC = 0.9799, G-Mean = 0.9417, MCC = 0.8622, F1-score = 0.8900
  Fold 9/10 - Experiment 10/10
Epoch 100/2000, Avg Loss: 0.

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 9:
      AdaBoost: AUC = 0.9016, G-Mean = 0.8797, MCC = 0.7911, F1-score = 0.8287
      SMOTEBoost: AUC = 0.9684, G-Mean = 0.9406, MCC = 0.8683, F1-score = 0.8957
      RUSBoost: AUC = 0.9648, G-Mean = 0.7774, MCC = 0.5805, F1-score = 0.6755
      OUBoost: AUC = 0.9811, G-Mean = 0.9171, MCC = 0.8207, F1-score = 0.8570
      SVM: AUC = 0.9821, G-Mean = 0.9187, MCC = 0.8537, F1-score = 0.8797
      SMOTE: AUC = 0.9821, G-Mean = 0.9231, MCC = 0.8477, F1-score = 0.8744
      ADASYN: AUC = 0.9848, G-Mean = 0.9398, MCC = 0.8755, F1-score = 0.9000
      bSMOTE: AUC = 0.9876, G-Mean = 0.9398, MCC = 0.8755, F1-score = 0.9000
      ROS: AUC = 0.9875, G-Mean = 0.9398, MCC = 0.8755, F1-score = 0.9000
      MWMOTE: AUC = 0.9861, G-Mean = 0.9515, MCC = 0.8902, F1-score = 0.9123
      Trans(Direct): AUC = 0.9821, G-Mean = 0.9482, MCC = 0.8775, F1-score = 0.9022
  Fold 10/10 - Experiment 10/10
Epoch 100/2000, Avg Loss: 0

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 10:
      AdaBoost: AUC = 0.8914, G-Mean = 0.8692, MCC = 0.7851, F1-score = 0.8209
      SMOTEBoost: AUC = 0.9716, G-Mean = 0.9465, MCC = 0.8815, F1-score = 0.9061
      RUSBoost: AUC = 0.9683, G-Mean = 0.7555, MCC = 0.5537, F1-score = 0.6555
      OUBoost: AUC = 0.9830, G-Mean = 0.9253, MCC = 0.8386, F1-score = 0.8713
      SVM: AUC = 0.9839, G-Mean = 0.9043, MCC = 0.8413, F1-score = 0.8668
      SMOTE: AUC = 0.9839, G-Mean = 0.9083, MCC = 0.8359, F1-score = 0.8620
      ADASYN: AUC = 0.9863, G-Mean = 0.9352, MCC = 0.8748, F1-score = 0.8988
      bSMOTE: AUC = 0.9876, G-Mean = 0.9352, MCC = 0.8748, F1-score = 0.8988
      ROS: AUC = 0.9887, G-Mean = 0.9352, MCC = 0.8748, F1-score = 0.8988
      MWMOTE: AUC = 0.9875, G-Mean = 0.9458, MCC = 0.8880, F1-score = 0.9100
      Trans(Direct): AUC = 0.9839, G-Mean = 0.9428, MCC = 0.8765, F1-score = 0.9009

Final Averaged Results Across Experiments:
  AdaBoost:
  

In [11]:
# yeast4
data_name = 'yeast4'
dataframe = pd.read_csv(f"{data_path}{data_name}.dat")
results = run_experiment(dataframe, device, h_dim=256, num_layers=10, beta=0.1, lr = 0.001, save_path=save_path, data_name = 'yeast4')

for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for metric, values in metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"  {metric}: {mean_value:.4f} ± {std_value:.4f}")

X shape: (1483, 8)
Class distribution: Counter({np.int64(0): 1432, np.int64(1): 51})

Starting experiment 1/10
  Fold 1/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.10894, Reg Loss: 0.18164
Epoch 200/2000, Avg Loss: 0.09434, Reg Loss: 0.15794
Epoch 300/2000, Avg Loss: 0.10512, Reg Loss: 0.16805
Epoch 400/2000, Avg Loss: 0.09314, Reg Loss: 0.15694
Epoch 500/2000, Avg Loss: 0.08485, Reg Loss: 0.15505
Epoch 600/2000, Avg Loss: 0.08077, Reg Loss: 0.15192
Epoch 700/2000, Avg Loss: 0.08315, Reg Loss: 0.15055
Epoch 800/2000, Avg Loss: 0.07507, Reg Loss: 0.15087
Epoch 900/2000, Avg Loss: 0.07536, Reg Loss: 0.15165
Epoch 1000/2000, Avg Loss: 0.07668, Reg Loss: 0.15399
Epoch 1100/2000, Avg Loss: 0.07400, Reg Loss: 0.14826
Epoch 1200/2000, Avg Loss: 0.07270, Reg Loss: 0.15083
Epoch 1300/2000, Avg Loss: 0.07291, Reg Loss: 0.15224
Epoch 1400/2000, Avg Loss: 0.07073, Reg Loss: 0.15006
Epoch 1500/2000, Avg Loss: 0.06803, Reg Loss: 0.14794
Epoch 1600/2000, Avg Loss: 0.06549, Reg Loss: 0.14768
Epoc

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 1:
      AdaBoost: AUC = 0.9569, G-Mean = 0.4425, MCC = 0.1996, F1-score = 0.2222
      SMOTEBoost: AUC = 0.7778, G-Mean = 0.4441, MCC = 0.2386, F1-score = 0.2500
      RUSBoost: AUC = 0.9132, G-Mean = 0.8233, MCC = 0.3071, F1-score = 0.2581
      OUBoost: AUC = 0.7674, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0649
      SVM: AUC = 0.7472, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.9028, G-Mean = 0.7274, MCC = 0.2546, F1-score = 0.2400
      ADASYN: AUC = 0.8917, G-Mean = 0.7303, MCC = 0.2640, F1-score = 0.2500
      bSMOTE: AUC = 0.9306, G-Mean = 0.7472, MCC = 0.3386, F1-score = 0.3333
      ROS: AUC = 0.9181, G-Mean = 0.8333, MCC = 0.3330, F1-score = 0.2857
      MWMOTE: AUC = 0.8986, G-Mean = 0.7360, MCC = 0.2848, F1-score = 0.2727
      Trans(Direct): AUC = 0.9417, G-Mean = 0.7360, MCC = 0.2848, F1-score = 0.2727
  Fold 2/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.1

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 2:
      AdaBoost: AUC = 0.9639, G-Mean = 0.6059, MCC = 0.3929, F1-score = 0.4111
      SMOTEBoost: AUC = 0.7889, G-Mean = 0.5361, MCC = 0.3344, F1-score = 0.3472
      RUSBoost: AUC = 0.8795, G-Mean = 0.7908, MCC = 0.3646, F1-score = 0.3433
      OUBoost: AUC = 0.8351, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0649
      SVM: AUC = 0.8681, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.9201, G-Mean = 0.7886, MCC = 0.3215, F1-score = 0.2939
      ADASYN: AUC = 0.9198, G-Mean = 0.8384, MCC = 0.3686, F1-score = 0.3250
      bSMOTE: AUC = 0.9479, G-Mean = 0.8050, MCC = 0.3947, F1-score = 0.3772
      ROS: AUC = 0.9389, G-Mean = 0.8399, MCC = 0.3543, F1-score = 0.3095
      MWMOTE: AUC = 0.9153, G-Mean = 0.7962, MCC = 0.3508, F1-score = 0.3268
      Trans(Direct): AUC = 0.9403, G-Mean = 0.7962, MCC = 0.3508, F1-score = 0.3268
  Fold 3/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.0

...

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 9:
      AdaBoost: AUC = 0.8888, G-Mean = 0.5076, MCC = 0.3701, F1-score = 0.3709
      SMOTEBoost: AUC = 0.8650, G-Mean = 0.5260, MCC = 0.3383, F1-score = 0.3500
      RUSBoost: AUC = 0.9000, G-Mean = 0.7234, MCC = 0.3588, F1-score = 0.3461
      OUBoost: AUC = 0.8437, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0666
      SVM: AUC = 0.8619, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.8998, G-Mean = 0.7609, MCC = 0.3148, F1-score = 0.2963
      ADASYN: AUC = 0.8975, G-Mean = 0.7918, MCC = 0.3273, F1-score = 0.2967
      bSMOTE: AUC = 0.8818, G-Mean = 0.6761, MCC = 0.3235, F1-score = 0.3201
      ROS: AUC = 0.9024, G-Mean = 0.7853, MCC = 0.3315, F1-score = 0.3058
      MWMOTE: AUC = 0.8861, G-Mean = 0.7308, MCC = 0.3067, F1-score = 0.2954
      Trans(Direct): AUC = 0.9096, G-Mean = 0.7740, MCC = 0.3611, F1-score = 0.3445
  Fold 10/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.

/home/oldrain123/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 10:
      AdaBoost: AUC = 0.8608, G-Mean = 0.4568, MCC = 0.3309, F1-score = 0.3338
      SMOTEBoost: AUC = 0.8535, G-Mean = 0.5175, MCC = 0.3217, F1-score = 0.3350
      RUSBoost: AUC = 0.8748, G-Mean = 0.7073, MCC = 0.3313, F1-score = 0.3223
      OUBoost: AUC = 0.8110, G-Mean = 0.0324, MCC = 0.0063, F1-score = 0.0672
      SVM: AUC = 0.8513, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.8879, G-Mean = 0.7435, MCC = 0.2965, F1-score = 0.2815
      ADASYN: AUC = 0.8837, G-Mean = 0.7704, MCC = 0.3056, F1-score = 0.2799
      bSMOTE: AUC = 0.8745, G-Mean = 0.6823, MCC = 0.3207, F1-score = 0.3166
      ROS: AUC = 0.8888, G-Mean = 0.7777, MCC = 0.3192, F1-score = 0.2946
      MWMOTE: AUC = 0.8759, G-Mean = 0.7169, MCC = 0.2905, F1-score = 0.2819
      Trans(Direct): AUC = 0.9085, G-Mean = 0.7684, MCC = 0.3480, F1-score = 0.3315

Starting experiment 2/10
  Fold 1/10 - Experiment 2/10
Ep

KeyboardInterrupt: 

In [ ]:
# glass4
data_name = 'glass4'
dataframe = pd.read_csv(f"{data_path}{data_name}.dat")
results = run_experiment(dataframe, device, h_dim=256, num_layers=10, beta=0.1, lr = 0.1, save_path=save_path, data_name = 'glass4')

for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for metric, values in metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"  {metric}: {mean_value:.4f} ± {std_value:.4f}")

X shape: (213, 9)
Class distribution: Counter({np.int64(0): 200, np.int64(1): 13})

Starting experiment 1/10
  Fold 1/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.23014, Reg Loss: 0.30246
Epoch 200/2000, Avg Loss: 0.24868, Reg Loss: 0.30516
Epoch 300/2000, Avg Loss: 0.21862, Reg Loss: 0.28338
Epoch 400/2000, Avg Loss: 0.23902, Reg Loss: 0.31693
Epoch 500/2000, Avg Loss: 0.20166, Reg Loss: 0.27800
Epoch 600/2000, Avg Loss: 0.20400, Reg Loss: 0.26491
Epoch 700/2000, Avg Loss: 0.18652, Reg Loss: 0.26249
Epoch 800/2000, Avg Loss: 0.20080, Reg Loss: 0.28091
Epoch 900/2000, Avg Loss: 0.20401, Reg Loss: 0.27937
Epoch 1000/2000, Avg Loss: 0.18883, Reg Loss: 0.28744
Epoch 1100/2000, Avg Loss: 0.19172, Reg Loss: 0.27041
Epoch 1200/2000, Avg Loss: 0.20076, Reg Loss: 0.27804
Epoch 1300/2000, Avg Loss: 0.19759, Reg Loss: 0.27031
Epoch 1400/2000, Avg Loss: 0.19554, Reg Loss: 0.27585
Epoch 1500/2000, Avg Loss: 0.19513, Reg Loss: 0.27948
Epoch 1600/2000, Avg Loss: 0.20281, Reg Loss: 0.28169
Epoch 

KeyboardInterrupt: 

In [ ]:
# wine
data_name = 'wine'
dataframe = pd.read_csv(f"{data_path}{data_name}.dat")
results = run_experiment(dataframe, device, h_dim=512, num_layers=10, beta=0.1, lr = 0.001, save_path=save_path, data_name = 'wine')

for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for metric, values in metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"  {metric}: {mean_value:.4f} ± {std_value:.4f}")

X shape: (1598, 11)
Class distribution: Counter({np.int64(0): 1545, np.int64(1): 53})

Starting experiment 1/10
  Fold 1/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.21844, Reg Loss: 0.29565
Epoch 200/2000, Avg Loss: 0.22965, Reg Loss: 0.30846
Epoch 300/2000, Avg Loss: 0.20432, Reg Loss: 0.29588
Epoch 400/2000, Avg Loss: 0.18249, Reg Loss: 0.29066
Epoch 500/2000, Avg Loss: 0.16899, Reg Loss: 0.28834
Epoch 600/2000, Avg Loss: 0.18095, Reg Loss: 0.29909
Epoch 700/2000, Avg Loss: 0.16101, Reg Loss: 0.29043
Epoch 800/2000, Avg Loss: 0.15472, Reg Loss: 0.28654
Epoch 900/2000, Avg Loss: 0.16060, Reg Loss: 0.29076
Epoch 1000/2000, Avg Loss: 0.15013, Reg Loss: 0.28531
Epoch 1100/2000, Avg Loss: 0.15133, Reg Loss: 0.28913
Epoch 1200/2000, Avg Loss: 0.14699, Reg Loss: 0.28186
Epoch 1300/2000, Avg Loss: 0.14976, Reg Loss: 0.28157
Epoch 1400/2000, Avg Loss: 0.14581, Reg Loss: 0.28416
Epoch 1500/2000, Avg Loss: 0.14302, Reg Loss: 0.28100
Epoch 1600/2000, Avg Loss: 0.13764, Reg Loss: 0.27899
Epo

/opt/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 1:
      AdaBoost: AUC = 0.9032, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTEBoost: AUC = 0.7194, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      RUSBoost: AUC = 0.7355, G-Mean = 0.6816, MCC = 0.1530, F1-score = 0.1395
      OUBoost: AUC = 0.6206, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SVM: AUC = 0.6452, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.9213, G-Mean = 0.8500, MCC = 0.3782, F1-score = 0.3333
      ADASYN: AUC = 0.9226, G-Mean = 0.8531, MCC = 0.3908, F1-score = 0.3478
      bSMOTE: AUC = 0.9161, G-Mean = 0.6180, MCC = 0.2680, F1-score = 0.2857
      ROS: AUC = 0.9252, G-Mean = 0.8591, MCC = 0.4191, F1-score = 0.3810
      MWMOTE: AUC = 0.9316, G-Mean = 0.7492, MCC = 0.3410, F1-score = 0.3333
      Trans(Direct): AUC = 0.9135, G-Mean = 0.8223, MCC = 0.2954, F1-score = 0.2424
  Fold 2/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.1

/opt/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 2:
      AdaBoost: AUC = 0.7397, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTEBoost: AUC = 0.5523, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      RUSBoost: AUC = 0.7258, G-Mean = 0.6120, MCC = 0.1031, F1-score = 0.1114
      OUBoost: AUC = 0.5003, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SVM: AUC = 0.7123, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.7290, G-Mean = 0.6314, MCC = 0.2017, F1-score = 0.2011
      ADASYN: AUC = 0.7316, G-Mean = 0.6337, MCC = 0.2098, F1-score = 0.2096
      bSMOTE: AUC = 0.7316, G-Mean = 0.5238, MCC = 0.1730, F1-score = 0.1984
      ROS: AUC = 0.7581, G-Mean = 0.6367, MCC = 0.2239, F1-score = 0.2262
      MWMOTE: AUC = 0.7232, G-Mean = 0.5833, MCC = 0.1888, F1-score = 0.2051
      Trans(Direct): AUC = 0.7948, G-Mean = 0.6151, MCC = 0.1552, F1-score = 0.1525
  Fold 3/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.2

/opt/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 3:
      AdaBoost: AUC = 0.7482, G-Mean = 0.1491, MCC = 0.1472, F1-score = 0.1111
      SMOTEBoost: AUC = 0.6260, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      RUSBoost: AUC = 0.7084, G-Mean = 0.5546, MCC = 0.1199, F1-score = 0.1349
      OUBoost: AUC = 0.5918, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SVM: AUC = 0.7497, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.7277, G-Mean = 0.5641, MCC = 0.1605, F1-score = 0.1711
      ADASYN: AUC = 0.7308, G-Mean = 0.6221, MCC = 0.1943, F1-score = 0.1977
      bSMOTE: AUC = 0.7578, G-Mean = 0.4949, MCC = 0.1565, F1-score = 0.1836
      ROS: AUC = 0.7578, G-Mean = 0.6234, MCC = 0.2013, F1-score = 0.2063
      MWMOTE: AUC = 0.7329, G-Mean = 0.5921, MCC = 0.1957, F1-score = 0.2108
      Trans(Direct): AUC = 0.7966, G-Mean = 0.5482, MCC = 0.1131, F1-score = 0.1255
  Fold 4/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.2

/opt/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 4:
      AdaBoost: AUC = 0.7368, G-Mean = 0.1118, MCC = 0.1104, F1-score = 0.0833
      SMOTEBoost: AUC = 0.5998, G-Mean = 0.1114, MCC = 0.0758, F1-score = 0.0714
      RUSBoost: AUC = 0.6413, G-Mean = 0.5184, MCC = 0.0945, F1-score = 0.1173
      OUBoost: AUC = 0.5669, G-Mean = 0.1111, MCC = 0.0600, F1-score = 0.0625
      SVM: AUC = 0.7310, G-Mean = 0.0000, MCC = 0.0000, F1-score = 0.0000
      SMOTE: AUC = 0.7313, G-Mean = 0.5690, MCC = 0.1505, F1-score = 0.1617
      ADASYN: AUC = 0.7306, G-Mean = 0.6125, MCC = 0.1759, F1-score = 0.1816
      bSMOTE: AUC = 0.7313, G-Mean = 0.4793, MCC = 0.1407, F1-score = 0.1689
      ROS: AUC = 0.7487, G-Mean = 0.6123, MCC = 0.1787, F1-score = 0.1860
      MWMOTE: AUC = 0.7490, G-Mean = 0.5943, MCC = 0.1896, F1-score = 0.2036
      Trans(Direct): AUC = 0.7758, G-Mean = 0.5537, MCC = 0.1082, F1-score = 0.1219
  Fold 5/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.2

KeyboardInterrupt: 

In [ ]:
# ecoli4
data_name = 'ecoli4'
dataframe = pd.read_csv(f"{data_path}{data_name}.dat")
results = run_experiment(dataframe, device='cpu', h_dim=256, num_layers=10, beta=0.1, lr = 0.01, save_path=save_path, data_name = 'ecoli4')

for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for metric, values in metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"  {metric}: {mean_value:.4f} ± {std_value:.4f}")

X shape: (335, 7)
Class distribution: Counter({np.int64(0): 313, np.int64(1): 22})

Starting experiment 1/10
  Fold 1/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.11124, Reg Loss: 0.18714
Epoch 200/2000, Avg Loss: 0.12892, Reg Loss: 0.19627
Epoch 300/2000, Avg Loss: 0.10946, Reg Loss: 0.18046
Epoch 400/2000, Avg Loss: 0.10211, Reg Loss: 0.18288
Epoch 500/2000, Avg Loss: 0.10903, Reg Loss: 0.18684
Epoch 600/2000, Avg Loss: 0.10382, Reg Loss: 0.18059
Epoch 700/2000, Avg Loss: 0.10442, Reg Loss: 0.18376
Epoch 800/2000, Avg Loss: 0.10542, Reg Loss: 0.18425
Epoch 900/2000, Avg Loss: 0.10481, Reg Loss: 0.18120
Epoch 1000/2000, Avg Loss: 0.10502, Reg Loss: 0.18415
Epoch 1100/2000, Avg Loss: 0.11223, Reg Loss: 0.18651
Epoch 1200/2000, Avg Loss: 0.10848, Reg Loss: 0.18338
Epoch 1300/2000, Avg Loss: 0.10274, Reg Loss: 0.17875
Epoch 1400/2000, Avg Loss: 0.10054, Reg Loss: 0.18149
Epoch 1500/2000, Avg Loss: 0.10885, Reg Loss: 0.18280
Epoch 1600/2000, Avg Loss: 0.11028, Reg Loss: 0.18229
Epoch 

/opt/anaconda3/envs/imb_clf/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


predict_proba
_compute_proba_from_decision
    Intermediate Fold Results for Fold 1:
      AdaBoost: AUC = 0.9297, G-Mean = 0.6731, MCC = 0.2967, F1-score = 0.3333
      SMOTEBoost: AUC = 0.9531, G-Mean = 0.6731, MCC = 0.2967, F1-score = 0.3333
      RUSBoost: AUC = 0.9609, G-Mean = 0.8660, MCC = 0.3873, F1-score = 0.3333
      OUBoost: AUC = 0.9531, G-Mean = 0.6731, MCC = 0.2967, F1-score = 0.3333
      SVM: AUC = 1.0000, G-Mean = 0.7071, MCC = 0.6963, F1-score = 0.6667
      SMOTE: AUC = 0.9844, G-Mean = 0.7071, MCC = 0.6963, F1-score = 0.6667
      ADASYN: AUC = 0.9531, G-Mean = 0.7071, MCC = 0.6963, F1-score = 0.6667
      bSMOTE: AUC = 0.9531, G-Mean = 0.7071, MCC = 0.6963, F1-score = 0.6667
      ROS: AUC = 0.9844, G-Mean = 0.7071, MCC = 0.6963, F1-score = 0.6667
      MWMOTE: AUC = 0.9844, G-Mean = 0.7071, MCC = 0.6963, F1-score = 0.6667
      Trans(Direct): AUC = 1.0000, G-Mean = 1.0000, MCC = 1.0000, F1-score = 1.0000
  Fold 2/10 - Experiment 1/10
Epoch 100/2000, Avg Loss: 0.1

KeyboardInterrupt: 